# Ch 9 Optimization Algorithms

Textbook: "Mathematical Methods for Artificial Intelligence" Chapter 9

This notebook covers unconstrained optimization problems and various optimization algorithms.
We primarily use quadratic functions to analyze algorithm characteristics and visualizations.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/your-repo/math4ai-notes/blob/main/Notebooks/ch09_Optimization_Algorithms.ipynb)

In [21]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
import time

# Fix random seed for reproducibility
np.random.seed(42)

# Font settings for plots
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.unicode_minus'] = False

# Plot style settings
plt.style.use('default')
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f']

## 9.1 Unconstrained Optimization

### 9.1.1 Definitions and Basic Concepts

**Classification of Minima:**
- **Global minimum**: $f(x^*) \leq f(x), \forall x \in \mathbb{R}^n$
- **Local minimum**: $\exists \delta > 0$ s.t. $f(x^*) \leq f(x), \forall x \in B(x^*, \delta)$
- **Strict local minimum**: $f(x^*) < f(x), \forall x \in B(x^*, \delta) \setminus \{x^*\}$
- **Isolated minimum**: Unique local minimum
- **Critical point**: $\nabla f(x^*) = 0$

### 9.1.2 Optimality Conditions

**First-order necessary condition**: If $x^*$ is a local minimum, then $\nabla f(x^*) = 0$

**Second-order necessary condition**: If $x^*$ is a local minimum, then $\nabla f(x^*) = 0$ and $\nabla^2 f(x^*) \succeq 0$ (PSD)

**Second-order sufficient condition**: If $\nabla f(x^*) = 0$ and $\nabla^2 f(x^*) \succ 0$ (PD), then $x^*$ is a strict local minimum

### 9.1.3 Analytical Solution for Quadratic Functions

For quadratic function $f(x) = \frac{1}{2}x^T Q x + b^T x + c$ (where $Q \succ 0$), the optimal solution is:
$$x^* = -Q^{-1}b$$

In [22]:
# Quadratic function definition and analytical vs numerical solution comparison

def create_quadratic_function(Q, b, c=0):
    """Create quadratic function using SPD matrix Q"""
    def f(x):
        x = np.array(x)
        return 0.5 * x.T @ Q @ x + b.T @ x + c
    
    def grad_f(x):
        x = np.array(x)
        return Q @ x + b
    
    def hess_f(x):
        return Q
    
    # Analytical optimal solution
    analytical_solution = -np.linalg.solve(Q, b)
    
    return f, grad_f, hess_f, analytical_solution

In [23]:
def numerical_gradient(f, x, h=1e-8):
    """Compute numerical gradient using finite differences"""
    x = np.array(x, dtype=float)
    grad = np.zeros_like(x)
    
    for i in range(len(x)):
        x_plus = x.copy()
        x_minus = x.copy()
        x_plus[i] += h
        x_minus[i] -= h
        grad[i] = (f(x_plus) - f(x_minus)) / (2 * h)
    
    return grad

def numerical_hessian(f, x, h=1e-5):
    """Compute numerical Hessian using finite differences"""
    x = np.array(x, dtype=float)
    n = len(x)
    hess = np.zeros((n, n))
    
    f_x = f(x)
    
    for i in range(n):
        for j in range(n):
            x_ij = x.copy()
            x_i = x.copy()
            x_j = x.copy()
            
            x_ij[i] += h
            x_ij[j] += h
            x_i[i] += h
            x_j[j] += h
            
            hess[i, j] = (f(x_ij) - f(x_i) - f(x_j) + f_x) / (h * h)
    
    return hess

In [24]:
# Create example quadratic functions with different condition numbers
print("9.1.3 Analytical vs Numerical Solutions for Quadratic Functions")
print("="*60)

# Function 1: Condition number 10
eigenvals1 = [1.0, 10.0]
Q1 = np.diag(eigenvals1)
b1 = np.array([2.0, -4.0])
f1, grad_f1, hess_f1, analytical_opt1 = create_quadratic_function(Q1, b1)

# Function 2: Condition number 1000  
eigenvals2 = [1.0, 1000.0]
Q2 = np.diag(eigenvals2)
b2 = np.array([3.0, -1000.0])
f2, grad_f2, hess_f2, analytical_opt2 = create_quadratic_function(Q2, b2)

print(f"Function 1: f(x) = 0.5*x^T*Q1*x + b1^T*x, cond(Q1) = {np.linalg.cond(Q1):.1f}")
print(f"Analytical optimum: x* = [{analytical_opt1[0]:.6f}, {analytical_opt1[1]:.6f}]")
print(f"Function value: f(x*) = {f1(analytical_opt1):.8f}")
print()

print(f"Function 2: f(x) = 0.5*x^T*Q2*x + b2^T*x, cond(Q2) = {np.linalg.cond(Q2):.1f}")  
print(f"Analytical optimum: x* = [{analytical_opt2[0]:.6f}, {analytical_opt2[1]:.6f}]")
print(f"Function value: f(x*) = {f2(analytical_opt2):.8f}")
print()

9.1.3 Analytical vs Numerical Solutions for Quadratic Functions
Function 1: f(x) = 0.5*x^T*Q1*x + b1^T*x, cond(Q1) = 10.0
Analytical optimum: x* = [-2.000000, 0.400000]
Function value: f(x*) = -2.80000000

Function 2: f(x) = 0.5*x^T*Q2*x + b2^T*x, cond(Q2) = 1000.0
Analytical optimum: x* = [-3.000000, 1.000000]
Function value: f(x*) = -504.50000000



In [25]:
# Numerical differentiation verification
test_point = np.array([1.0, 2.0])
print("Numerical Differentiation Verification (Test point: [1.0, 2.0])")
print("-" * 30)

analytical_grad1 = grad_f1(test_point)
numerical_grad1 = numerical_gradient(f1, test_point)
print(f"Function 1 - Analytical gradient: [{analytical_grad1[0]:.6f}, {analytical_grad1[1]:.6f}]")
print(f"Function 1 - Numerical gradient:  [{numerical_grad1[0]:.6f}, {numerical_grad1[1]:.6f}]")
print(f"Gradient error: {np.linalg.norm(analytical_grad1 - numerical_grad1):.2e}")
print()

analytical_hess1 = hess_f1(test_point)
numerical_hess1 = numerical_hessian(f1, test_point)
print(f"Function 1 - Analytical Hessian:")
print(analytical_hess1)
print(f"Function 1 - Numerical Hessian:")
print(numerical_hess1)
print(f"Hessian error: {np.linalg.norm(analytical_hess1 - numerical_hess1):.2e}")
print()

Numerical Differentiation Verification (Test point: [1.0, 2.0])
------------------------------
Function 1 - Analytical gradient: [3.000000, 16.000000]
Function 1 - Numerical gradient:  [3.000000, 16.000000]
Gradient error: 1.07e-07

Function 1 - Analytical Hessian:
[[ 1.  0.]
 [ 0. 10.]]
Function 1 - Numerical Hessian:
[[1.00000008e+00 1.77635684e-05]
 [1.77635684e-05 9.99996530e+00]]
Hessian error: 4.28e-05



In [26]:
# Optimality conditions verification
print("Optimality Conditions Verification")
print("-" * 20)
grad_at_opt1 = grad_f1(analytical_opt1)
hess_at_opt1 = hess_f1(analytical_opt1)
eigenvals_hess1 = np.linalg.eigvals(hess_at_opt1)

print(f"Gradient norm at optimum: ||∇f(x*)|| = {np.linalg.norm(grad_at_opt1):.2e}")
print(f"Hessian eigenvalues: {eigenvals_hess1}")
print(f"Is Hessian positive definite? {np.all(eigenvals_hess1 > 0)}")

Optimality Conditions Verification
--------------------
Gradient norm at optimum: ||∇f(x*)|| = 0.00e+00
Hessian eigenvalues: [ 1. 10.]
Is Hessian positive definite? True


In [27]:
# Basic optimization algorithms implementation

def gradient_descent(f, grad_f, x0, learning_rate=0.01, max_iter=1000, tol=1e-6):
    """Gradient descent algorithm"""
    x = x0.copy()
    history = {
        'x': [x.copy()], 
        'f': [f(x)], 
        'grad_norm': [np.linalg.norm(grad_f(x))],
        'step_size': [],
        'iteration': [0],
        'time': [0]
    }
    
    start_time = time.time()
    
    for i in range(max_iter):
        grad = grad_f(x)
        grad_norm = np.linalg.norm(grad)
        
        if grad_norm < tol:
            break
            
        x_new = x - learning_rate * grad
        
        current_time = time.time() - start_time
        history['x'].append(x_new.copy())
        history['f'].append(f(x_new))
        history['grad_norm'].append(np.linalg.norm(grad_f(x_new)))
        history['step_size'].append(learning_rate)
        history['iteration'].append(i+1)
        history['time'].append(current_time)
        
        x = x_new
    
    return x, history

In [28]:
def newton_method(f, grad_f, hess_f, x0, max_iter=100, tol=1e-6):
    """Newton's method for optimization"""
    x = x0.copy()
    history = {
        'x': [x.copy()], 
        'f': [f(x)], 
        'grad_norm': [np.linalg.norm(grad_f(x))],
        'step_size': [],
        'iteration': [0],
        'time': [0]
    }
    
    start_time = time.time()
    
    for i in range(max_iter):
        grad = grad_f(x)
        grad_norm = np.linalg.norm(grad)
        
        if grad_norm < tol:
            break
            
        hess = hess_f(x)
        try:
            step = np.linalg.solve(hess, grad)
            x_new = x - step
            step_size = np.linalg.norm(step)
        except np.linalg.LinAlgError:
            step = 0.01 * grad
            x_new = x - step
            step_size = 0.01 * grad_norm
        
        current_time = time.time() - start_time
        history['x'].append(x_new.copy())
        history['f'].append(f(x_new))
        history['grad_norm'].append(np.linalg.norm(grad_f(x_new)))
        history['step_size'].append(step_size)
        history['iteration'].append(i+1)
        history['time'].append(current_time)
        
        x = x_new
    
    return x, history

In [29]:
# Analytical vs Numerical solution comparison experiment
print("Analytical vs Gradient Descent Comparison")
print("="*40)

# Starting point
x0 = np.array([5.0, -3.0])

# Gradient descent on function 1
x_gd1, hist_gd1 = gradient_descent(f1, grad_f1, x0, learning_rate=0.1, max_iter=200)
print(f"\nCondition number {np.linalg.cond(Q1):.1f} function:")
print(f"Analytical optimum:  x* = [{analytical_opt1[0]:.6f}, {analytical_opt1[1]:.6f}]")
print(f"Gradient descent result: x* = [{x_gd1[0]:.6f}, {x_gd1[1]:.6f}]")
print(f"Error: {np.linalg.norm(x_gd1 - analytical_opt1):.2e}")
print(f"Iterations: {len(hist_gd1['f'])-1}")

# Gradient descent on function 2
x_gd2, hist_gd2 = gradient_descent(f2, grad_f2, x0, learning_rate=0.001, max_iter=2000)
print(f"\nCondition number {np.linalg.cond(Q2):.1f} function:")
print(f"Analytical optimum:  x* = [{analytical_opt2[0]:.6f}, {analytical_opt2[1]:.6f}]")
print(f"Gradient descent result: x* = [{x_gd2[0]:.6f}, {x_gd2[1]:.6f}]")
print(f"Error: {np.linalg.norm(x_gd2 - analytical_opt2):.2e}")
print(f"Iterations: {len(hist_gd2['f'])-1}")

Analytical vs Gradient Descent Comparison

Condition number 10.0 function:
Analytical optimum:  x* = [-2.000000, 0.400000]
Gradient descent result: x* = [-1.999999, 0.400000]
Error: 9.58e-07
Iterations: 150

Condition number 1000.0 function:
Analytical optimum:  x* = [-3.000000, 1.000000]
Gradient descent result: x* = [-1.918401, 1.000000]
Error: 1.08e+00
Iterations: 2000


## 9.2 Line Search Methods

### 9.2.1 Learning Rate and Convergence Speed

The choice of learning rate (step size) $\alpha$ in gradient descent significantly affects convergence speed.

### 9.2.2 Wolfe and Armijo Conditions

**Armijo condition** (sufficient decrease):
$$f(x_k + \alpha_k d_k) \leq f(x_k) + c_1 \alpha_k \nabla f(x_k)^T d_k$$

**Wolfe conditions** (adds curvature condition):
$$|\nabla f(x_k + \alpha_k d_k)^T d_k| \leq c_2 |\nabla f(x_k)^T d_k|$$

where $0 < c_1 < c_2 < 1$.

In [30]:
def armijo_line_search(f, grad_f, x, direction, c1=1e-4, alpha_init=1.0, rho=0.5, max_iter=50):
    """
    Armijo backtracking line search
    
    Parameters:
    - f: objective function
    - grad_f: gradient function
    - x: current point
    - direction: search direction
    - c1: Armijo constant (default: 1e-4)
    - alpha_init: initial step size (default: 1.0)
    - rho: reduction factor (default: 0.5)
    - max_iter: maximum iterations (default: 50)
    
    Returns:
    - alpha: step size satisfying Armijo condition
    """
    alpha = alpha_init
    fx = f(x)
    grad_fx = grad_f(x)
    
    for i in range(max_iter):
        if f(x + alpha * direction) <= fx + c1 * alpha * np.dot(grad_fx, direction):
            return alpha
        alpha *= rho
    
    return alpha

In [31]:
def wolfe_line_search(f, grad_f, x, direction, c1=1e-4, c2=0.9, alpha_init=1.0, max_iter=50):
    """
    Wolfe line search (strong Wolfe conditions)
    
    Parameters:
    - f: objective function
    - grad_f: gradient function
    - x: current point
    - direction: search direction
    - c1: Armijo constant (default: 1e-4)
    - c2: curvature constant (default: 0.9)
    - alpha_init: initial step size (default: 1.0)
    - max_iter: maximum iterations (default: 50)
    
    Returns:
    - alpha: step size satisfying Wolfe conditions
    """
    def zoom(alpha_lo, alpha_hi):
        for _ in range(max_iter):
            alpha = (alpha_lo + alpha_hi) / 2
            x_new = x + alpha * direction
            
            if f(x_new) > fx + c1 * alpha * grad_fx_dot_d or f(x_new) >= f(x + alpha_lo * direction):
                alpha_hi = alpha
            else:
                grad_new = grad_f(x_new)
                if abs(np.dot(grad_new, direction)) <= -c2 * grad_fx_dot_d:
                    return alpha
                if np.dot(grad_new, direction) * (alpha_hi - alpha_lo) >= 0:
                    alpha_hi = alpha_lo
                alpha_lo = alpha
        return alpha
    
    alpha = alpha_init
    fx = f(x)
    grad_fx = grad_f(x)
    grad_fx_dot_d = np.dot(grad_fx, direction)
    
    alpha_prev = 0
    for i in range(1, max_iter + 1):
        x_new = x + alpha * direction
        fx_new = f(x_new)
        
        if fx_new > fx + c1 * alpha * grad_fx_dot_d or (i > 1 and fx_new >= f(x + alpha_prev * direction)):
            return zoom(alpha_prev, alpha)
        
        grad_new = grad_f(x_new)
        if abs(np.dot(grad_new, direction)) <= -c2 * grad_fx_dot_d:
            return alpha
        
        if np.dot(grad_new, direction) >= 0:
            return zoom(alpha, alpha_prev)
        
        alpha_prev = alpha
        alpha *= 2
    
    return alpha

In [ ]:
# Test line search methods with visualization
def test_line_search_methods():
    """Test and visualize line search methods"""
    # Create a simple quadratic function for testing
    Q_test = np.array([[2.0, 0.5], [0.5, 1.0]])
    b_test = np.array([1.0, 1.0])
    f_test, grad_f_test, _, _ = create_quadratic_function(Q_test, b_test)
    
    x_test = np.array([2.0, 2.0])
    direction_test = -grad_f_test(x_test)  # Steepest descent direction
    
    # Test both methods
    alpha_armijo = armijo_line_search(f_test, grad_f_test, x_test, direction_test)
    alpha_wolfe = wolfe_line_search(f_test, grad_f_test, x_test, direction_test)
    
    print(f"Starting point: {x_test}")
    print(f"Search direction: {direction_test}")
    print(f"Armijo step size: {alpha_armijo:.6f}")
    print(f"Wolfe step size: {alpha_wolfe:.6f}")
    
    # Visualize line search
    alphas = np.linspace(0, 2, 100)
    phi_values = [f_test(x_test + alpha * direction_test) for alpha in alphas]
    
    plt.figure(figsize=(10, 6))
    plt.plot(alphas, phi_values, 'b-', label='φ(α) = f(x + α·d)', linewidth=2)
    plt.axvline(x=alpha_armijo, color='red', linestyle='--', label=f'Armijo: α = {alpha_armijo:.4f}')
    plt.axvline(x=alpha_wolfe, color='green', linestyle='--', label=f'Wolfe: α = {alpha_wolfe:.4f}')
    plt.xlabel('Step size α')
    plt.ylabel('Function value φ(α)')
    plt.title('Line Search Methods Comparison')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()

# Run the test
test_line_search_methods()

In [ ]:
def gradient_descent_with_line_search(f, grad_f, x0, line_search_method='armijo', max_iter=1000, tol=1e-6):
    """Gradient descent with line search"""
    x = x0.copy()
    history = {
        'x': [x.copy()], 
        'f': [f(x)], 
        'grad_norm': [np.linalg.norm(grad_f(x))],
        'step_size': [],
        'iteration': [0],
        'time': [0]
    }
    
    start_time = time.time()
    
    for i in range(max_iter):
        grad = grad_f(x)
        grad_norm = np.linalg.norm(grad)
        
        if grad_norm < tol:
            break
            
        direction = -grad  # Steepest descent direction
        
        # Choose line search method
        if line_search_method == 'armijo':
            alpha = armijo_line_search(f, grad_f, x, direction)
        elif line_search_method == 'wolfe':
            alpha = wolfe_line_search(f, grad_f, x, direction)
        else:
            alpha = 0.01  # Fixed step size fallback
            
        x_new = x + alpha * direction
        
        current_time = time.time() - start_time
        history['x'].append(x_new.copy())
        history['f'].append(f(x_new))
        history['grad_norm'].append(np.linalg.norm(grad_f(x_new)))
        history['step_size'].append(alpha)
        history['iteration'].append(i+1)
        history['time'].append(current_time)
        
        x = x_new
    
    return x, history

## 9.3 Optimization Algorithms

### 9.3.1 BFGS and L-BFGS

**BFGS (Broyden–Fletcher–Goldfarb–Shanno)**: A quasi-Newton method that iteratively updates an approximation of the Hessian using gradient information

**L-BFGS (Limited-memory BFGS)**: A memory-efficient variant of BFGS that uses a two-loop recursion

### 9.3.2 Momentum and Adaptive Methods

**Momentum**: Uses inertia from previous update directions to reduce oscillations and accelerate convergence

**Nesterov momentum**: An improved momentum method that considers the gradient at the anticipated future position

**Adam**: Combines momentum with adaptive learning rates for robust optimization

In [ ]:
# 9.3 Optimization algorithms implementation

def bfgs(f, grad_f, x0, max_iter=1000, tol=1e-6):
    """BFGS quasi-Newton method"""
    x = x0.copy()
    n = len(x)
    B_inv = np.eye(n)  # Initialize inverse Hessian approximation
    
    history = {
        'x': [x.copy()], 
        'f': [f(x)], 
        'grad_norm': [np.linalg.norm(grad_f(x))],
        'step_size': [],
        'iteration': [0],
        'time': [0]
    }
    
    start_time = time.time()
    grad = grad_f(x)
    
    for i in range(max_iter):
        grad_norm = np.linalg.norm(grad)
        if grad_norm < tol:
            break
        
        # Search direction
        direction = -B_inv @ grad
        
        # Line search
        alpha = armijo_line_search(f, grad_f, x, direction)
        
        # Update
        x_new = x + alpha * direction
        grad_new = grad_f(x_new)
        
        # BFGS update
        s = x_new - x
        y = grad_new - grad
        
        # Check curvature condition
        if np.dot(s, y) > 1e-10:
            rho = 1 / np.dot(y, s)
            I = np.eye(n)
            B_inv = (I - rho * np.outer(s, y)) @ B_inv @ (I - rho * np.outer(y, s)) + rho * np.outer(s, s)
        
        current_time = time.time() - start_time
        history['x'].append(x_new.copy())
        history['f'].append(f(x_new))
        history['grad_norm'].append(np.linalg.norm(grad_new))
        history['step_size'].append(alpha)
        history['iteration'].append(i+1)
        history['time'].append(current_time)
        
        x = x_new
        grad = grad_new
    
    return x, history

In [ ]:
def lbfgs_two_loop(s_list, y_list, grad, m=10):
    """L-BFGS two-loop recursion"""
    q = grad.copy()
    alpha_list = []
    rho_list = []
    
    # First loop
    for i in range(len(s_list)-1, -1, -1):
        s_i = s_list[i]
        y_i = y_list[i]
        rho_i = 1 / np.dot(y_i, s_i)
        rho_list.append(rho_i)
        alpha_i = rho_i * np.dot(s_i, q)
        alpha_list.append(alpha_i)
        q = q - alpha_i * y_i
    
    # Scaling
    if len(s_list) > 0:
        s_k = s_list[-1]
        y_k = y_list[-1]
        gamma = np.dot(s_k, y_k) / np.dot(y_k, y_k)
        r = gamma * q
    else:
        r = q
    
    # Second loop
    rho_list.reverse()
    alpha_list.reverse()
    
    for i in range(len(s_list)):
        s_i = s_list[i]
        y_i = y_list[i]
        rho_i = rho_list[i]
        alpha_i = alpha_list[i]
        beta = rho_i * np.dot(y_i, r)
        r = r + s_i * (alpha_i - beta)
    
    return r

In [ ]:
def lbfgs(f, grad_f, x0, m=10, max_iter=1000, tol=1e-6):
    """L-BFGS method"""
    x = x0.copy()
    s_list = []
    y_list = []
    
    history = {
        'x': [x.copy()], 
        'f': [f(x)], 
        'grad_norm': [np.linalg.norm(grad_f(x))],
        'step_size': [],
        'iteration': [0],
        'time': [0]
    }
    
    start_time = time.time()
    grad = grad_f(x)
    
    for i in range(max_iter):
        grad_norm = np.linalg.norm(grad)
        if grad_norm < tol:
            break
        
        # L-BFGS direction computation
        if len(s_list) == 0:
            direction = -grad
        else:
            direction = -lbfgs_two_loop(s_list, y_list, grad, m)
        
        # Line search
        alpha = armijo_line_search(f, grad_f, x, direction)
        
        # Update
        x_new = x + alpha * direction
        grad_new = grad_f(x_new)
        
        # Store history
        s = x_new - x
        y = grad_new - grad
        
        if np.dot(s, y) > 1e-10:
            s_list.append(s)
            y_list.append(y)
            
            # Memory limit
            if len(s_list) > m:
                s_list.pop(0)
                y_list.pop(0)
        
        current_time = time.time() - start_time
        history['x'].append(x_new.copy())
        history['f'].append(f(x_new))
        history['grad_norm'].append(np.linalg.norm(grad_new))
        history['step_size'].append(alpha)
        history['iteration'].append(i+1)
        history['time'].append(current_time)
        
        x = x_new
        grad = grad_new
    
    return x, history

In [ ]:
def momentum(f, grad_f, x0, learning_rate=0.01, beta=0.9, max_iter=1000, tol=1e-6):
    """Momentum gradient descent"""
    x = x0.copy()
    v = np.zeros_like(x)
    
    history = {
        'x': [x.copy()], 
        'f': [f(x)], 
        'grad_norm': [np.linalg.norm(grad_f(x))],
        'step_size': [],
        'iteration': [0],
        'time': [0]
    }
    
    start_time = time.time()
    
    for i in range(max_iter):
        grad = grad_f(x)
        grad_norm = np.linalg.norm(grad)
        
        if grad_norm < tol:
            break
        
        v = beta * v + learning_rate * grad
        x_new = x - v
        
        current_time = time.time() - start_time
        history['x'].append(x_new.copy())
        history['f'].append(f(x_new))
        history['grad_norm'].append(np.linalg.norm(grad_f(x_new)))
        history['step_size'].append(np.linalg.norm(v))
        history['iteration'].append(i+1)
        history['time'].append(current_time)
        
        x = x_new
    
    return x, history

In [ ]:
def nesterov(f, grad_f, x0, learning_rate=0.01, beta=0.9, max_iter=1000, tol=1e-6):
    """Nesterov accelerated gradient descent"""
    x = x0.copy()
    v = np.zeros_like(x)
    
    history = {
        'x': [x.copy()], 
        'f': [f(x)], 
        'grad_norm': [np.linalg.norm(grad_f(x))],
        'step_size': [],
        'iteration': [0],
        'time': [0]
    }
    
    start_time = time.time()
    
    for i in range(max_iter):
        # Gradient at anticipated future position
        x_future = x - beta * v
        grad = grad_f(x_future)
        grad_norm = np.linalg.norm(grad_f(x))
        
        if grad_norm < tol:
            break
        
        v = beta * v + learning_rate * grad
        x_new = x - v
        
        current_time = time.time() - start_time
        history['x'].append(x_new.copy())
        history['f'].append(f(x_new))
        history['grad_norm'].append(np.linalg.norm(grad_f(x_new)))
        history['step_size'].append(np.linalg.norm(v))
        history['iteration'].append(i+1)
        history['time'].append(current_time)
        
        x = x_new
    
    return x, history

In [ ]:
# Algorithm comparison on quadratic functions
print("9.3 Optimization Algorithms Comparison (Quadratic Functions)")
print("="*50)

x0 = np.array([4.0, -3.0])

# Verify all required functions are available
try:
    # Test line search functions
    test_direction = np.array([-1.0, 0.0])
    armijo_line_search(f1, grad_f1, x0, test_direction)
    print("Line search functions are available")
    
    # BFGS vs L-BFGS comparison (well-conditioned function)
    print(f"\nQuasi-Newton methods comparison on condition number {np.linalg.cond(Q1):.1f} function:")
    x_bfgs, hist_bfgs = bfgs(f1, grad_f1, x0, max_iter=50)
    x_lbfgs, hist_lbfgs = lbfgs(f1, grad_f1, x0, max_iter=50)

    print(f"  BFGS:   Final error {np.linalg.norm(x_bfgs - analytical_opt1):.2e}, Iterations {len(hist_bfgs['f'])-1:3d}")
    print(f"  L-BFGS: Final error {np.linalg.norm(x_lbfgs - analytical_opt1):.2e}, Iterations {len(hist_lbfgs['f'])-1:3d}")

    # Momentum methods comparison
    print(f"\nMomentum methods comparison on condition number {np.linalg.cond(Q1):.1f} function:")
    x_gd, hist_gd = gradient_descent(f1, grad_f1, x0, learning_rate=0.1, max_iter=100)
    x_mom, hist_mom = momentum(f1, grad_f1, x0, learning_rate=0.1, beta=0.9, max_iter=100)  
    x_nest, hist_nest = nesterov(f1, grad_f1, x0, learning_rate=0.1, beta=0.9, max_iter=100)

    print(f"  GD:        Final error {np.linalg.norm(x_gd - analytical_opt1):.2e}, Iterations {len(hist_gd['f'])-1:3d}")
    print(f"  Momentum:  Final error {np.linalg.norm(x_mom - analytical_opt1):.2e}, Iterations {len(hist_mom['f'])-1:3d}")
    print(f"  Nesterov:  Final error {np.linalg.norm(x_nest - analytical_opt1):.2e}, Iterations {len(hist_nest['f'])-1:3d}")

    # Store results for visualization
    results_quadratic = {
        'BFGS': (x_bfgs, hist_bfgs),
        'L-BFGS': (x_lbfgs, hist_lbfgs),
        'GD': (x_gd, hist_gd),
        'Momentum': (x_mom, hist_mom),
        'Nesterov': (x_nest, hist_nest)
    }
    
    print("\nAlgorithm comparison completed successfully")
    
except NameError as e:
    print(f"Error: Missing function - {e}")
    print("Please ensure all previous cells have been executed in order.")
    # Create empty results for safety
    results_quadratic = {}

In [ ]:
# Adaptive learning rate methods and logistic regression experiments

def adagrad(f, grad_f, x0, learning_rate=0.01, eps=1e-8, max_iter=1000, tol=1e-6):
    """Adagrad algorithm"""
    x = x0.copy()
    G = np.zeros_like(x)
    
    history = {
        'x': [x.copy()], 
        'f': [f(x)], 
        'grad_norm': [np.linalg.norm(grad_f(x))],
        'step_size': [],
        'iteration': [0],
        'time': [0]
    }
    
    start_time = time.time()
    
    for i in range(max_iter):
        grad = grad_f(x)
        grad_norm = np.linalg.norm(grad)
        
        if grad_norm < tol:
            break
        
        G += grad * grad
        adapted_lr = learning_rate / (np.sqrt(G) + eps)
        x_new = x - adapted_lr * grad
        
        current_time = time.time() - start_time
        history['x'].append(x_new.copy())
        history['f'].append(f(x_new))
        history['grad_norm'].append(np.linalg.norm(grad_f(x_new)))
        history['step_size'].append(np.mean(adapted_lr))
        history['iteration'].append(i+1)
        history['time'].append(current_time)
        
        x = x_new
    
    return x, history

In [ ]:
def rmsprop(f, grad_f, x0, learning_rate=0.01, beta=0.9, eps=1e-8, max_iter=1000, tol=1e-6):
    """RMSProp algorithm"""
    x = x0.copy()
    v = np.zeros_like(x)
    
    history = {
        'x': [x.copy()], 
        'f': [f(x)], 
        'grad_norm': [np.linalg.norm(grad_f(x))],
        'step_size': [],
        'iteration': [0],
        'time': [0]
    }
    
    start_time = time.time()
    
    for i in range(max_iter):
        grad = grad_f(x)
        grad_norm = np.linalg.norm(grad)
        
        if grad_norm < tol:
            break
        
        v = beta * v + (1 - beta) * grad * grad
        adapted_lr = learning_rate / (np.sqrt(v) + eps)
        x_new = x - adapted_lr * grad
        
        current_time = time.time() - start_time
        history['x'].append(x_new.copy())
        history['f'].append(f(x_new))
        history['grad_norm'].append(np.linalg.norm(grad_f(x_new)))
        history['step_size'].append(np.mean(adapted_lr))
        history['iteration'].append(i+1)
        history['time'].append(current_time)
        
        x = x_new
    
    return x, history

def adadelta(f, grad_f, x0, beta=0.95, eps=1e-6, max_iter=1000, tol=1e-6):
    """Adadelta algorithm"""
    x = x0.copy()
    v = np.zeros_like(x)
    u = np.zeros_like(x)
    
    history = {
        'x': [x.copy()], 
        'f': [f(x)], 
        'grad_norm': [np.linalg.norm(grad_f(x))],
        'step_size': [],
        'iteration': [0],
        'time': [0]
    }
    
    start_time = time.time()
    
    for i in range(max_iter):
        grad = grad_f(x)
        grad_norm = np.linalg.norm(grad)
        
        if grad_norm < tol:
            break
        
        v = beta * v + (1 - beta) * grad * grad
        delta_x = -np.sqrt(u + eps) / np.sqrt(v + eps) * grad
        u = beta * u + (1 - beta) * delta_x * delta_x
        x_new = x + delta_x
        
        current_time = time.time() - start_time
        history['x'].append(x_new.copy())
        history['f'].append(f(x_new))
        history['grad_norm'].append(np.linalg.norm(grad_f(x_new)))
        history['step_size'].append(np.linalg.norm(delta_x))
        history['iteration'].append(i+1)
        history['time'].append(current_time)
        
        x = x_new
    
    return x, history

def adam(f, grad_f, x0, learning_rate=0.001, beta1=0.9, beta2=0.999, eps=1e-8, max_iter=1000, tol=1e-6):
    """Adam algorithm"""
    x = x0.copy()
    m = np.zeros_like(x)
    v = np.zeros_like(x)
    
    history = {
        'x': [x.copy()], 
        'f': [f(x)], 
        'grad_norm': [np.linalg.norm(grad_f(x))],
        'step_size': [],
        'iteration': [0],
        'time': [0]
    }
    
    start_time = time.time()
    
    for i in range(max_iter):
        grad = grad_f(x)
        grad_norm = np.linalg.norm(grad)
        
        if grad_norm < tol:
            break
        
        t = i + 1
        m = beta1 * m + (1 - beta1) * grad
        v = beta2 * v + (1 - beta2) * grad * grad
        
        m_hat = m / (1 - beta1**t)
        v_hat = v / (1 - beta2**t)
        
        step = learning_rate * m_hat / (np.sqrt(v_hat) + eps)
        x_new = x - step
        
        current_time = time.time() - start_time
        history['x'].append(x_new.copy())
        history['f'].append(f(x_new))
        history['grad_norm'].append(np.linalg.norm(grad_f(x_new)))
        history['step_size'].append(np.linalg.norm(step))
        history['iteration'].append(i+1)
        history['time'].append(current_time)
        
        x = x_new
    
    return x, history

In [ ]:
# Synthetic logistic regression data generation (small dataset)
def create_logistic_regression_data(n_samples=100, n_features=5):
    """Generate synthetic data for logistic regression"""
    np.random.seed(42)
    X = np.random.randn(n_samples, n_features)
    true_w = np.random.randn(n_features)
    y = (X @ true_w + 0.1 * np.random.randn(n_samples) > 0).astype(float)
    return X, y, true_w

def logistic_loss_and_grad(w, X, y):
    """Logistic loss function and gradient"""
    z = X @ w
    # Clipping for numerical stability
    z = np.clip(z, -500, 500)
    
    # Logistic loss
    loss = np.mean(np.log(1 + np.exp(-y * z)))
    
    # Gradient 
    sigmoid = 1 / (1 + np.exp(-z))
    grad = -X.T @ (y * (1 - sigmoid)) / len(y)
    
    return loss, grad

# Generate logistic regression data
X, y, true_w = create_logistic_regression_data(100, 5)
y = 2 * y - 1  # {0,1} -> {-1,1}

def logistic_f(w):
    loss, _ = logistic_loss_and_grad(w, X, y)
    return loss

def logistic_grad_f(w):
    _, grad = logistic_loss_and_grad(w, X, y)
    return grad

In [ ]:
# Adaptive learning rate methods comparison (logistic regression)
print("\nAdaptive Learning Rate Methods Comparison (Logistic Regression)")
print("="*50)

w0 = np.random.randn(5) * 0.1
print(f"Initial loss: {logistic_f(w0):.6f}")

# Verify adaptive functions are available
try:
    # Test functions exist
    test_grad = np.array([0.1, 0.1, 0.1, 0.1, 0.1])
    adagrad(lambda x: np.sum(x**2), lambda x: 2*x, w0*0.1, max_iter=1)
    print("Adaptive learning rate functions are available")
    
    adaptive_methods = [
        ('Adagrad', lambda: adagrad(logistic_f, logistic_grad_f, w0, learning_rate=0.1, max_iter=500)),
        ('RMSProp', lambda: rmsprop(logistic_f, logistic_grad_f, w0, learning_rate=0.01, max_iter=500)),  
        ('Adadelta', lambda: adadelta(logistic_f, logistic_grad_f, w0, max_iter=500)),
        ('Adam', lambda: adam(logistic_f, logistic_grad_f, w0, learning_rate=0.01, max_iter=500)),
    ]

    results_adaptive = {}
    for name, method in adaptive_methods:
        w_opt, history = method()
        results_adaptive[name] = (w_opt, history)
        final_loss = logistic_f(w_opt)
        print(f"  {name:10}: Final loss {final_loss:.6f}, Iterations {len(history['f'])-1:3d}")

    print(f"\nReference - True weights: {true_w}")
    for name, (w_opt, _) in results_adaptive.items():
        error = np.linalg.norm(w_opt - true_w)
        print(f"  {name:10}: Weight error {error:.6f}")
        
    print("\nAdaptive learning rate comparison completed successfully")
    
except NameError as e:
    print(f"Error: Missing function - {e}")
    print("Please ensure all previous cells have been executed in order.")
    # Create empty results for safety
    results_adaptive = {}

In [ ]:
# Visualization 3: Adaptive learning rate methods comparison

def plot_adaptive_methods_comparison():
    """Visualize adaptive learning rate methods performance"""
    
    # Check if results are available
    if 'results_adaptive' not in globals() or not results_adaptive:
        print("Error: Adaptive algorithm results not available. Please run the adaptive methods comparison cell first.")
        return
    
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    axes = axes.flatten()
    
    # Plot 0: Loss convergence comparison
    ax = axes[0]
    colors_adaptive = ['red', 'blue', 'green', 'purple']
    
    for i, (method_name, (w_opt, hist)) in enumerate(results_adaptive.items()):
        loss_values = hist['f']
        ax.plot(loss_values, color=colors_adaptive[i], linewidth=2, 
                label=method_name, alpha=0.8)
    
    ax.set_xlabel('Iteration')
    ax.set_ylabel('Loss')
    ax.set_title('Loss Convergence: Adaptive Methods')
    ax.legend()
    ax.grid(True, alpha=0.3)
    ax.set_yscale('log')
    
    # Plot 1: Gradient norm evolution
    ax = axes[1]
    
    for i, (method_name, (w_opt, hist)) in enumerate(results_adaptive.items()):
        grad_norms = hist['grad_norm']
        ax.semilogy(grad_norms, color=colors_adaptive[i], linewidth=2, 
                   label=method_name, alpha=0.8)
    
    ax.set_xlabel('Iteration')
    ax.set_ylabel('||∇f(w)||')
    ax.set_title('Gradient Norm Evolution')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # Plot 2: Step size evolution
    ax = axes[2]
    
    for i, (method_name, (w_opt, hist)) in enumerate(results_adaptive.items()):
        step_sizes = hist['step_size']
        if step_sizes and len(step_sizes) > 1:
            ax.plot(step_sizes, color=colors_adaptive[i], linewidth=2, 
                   label=method_name, alpha=0.8)
    
    ax.set_xlabel('Iteration')
    ax.set_ylabel('Step Size')
    ax.set_title('Adaptive Step Size Evolution')
    ax.legend()
    ax.grid(True, alpha=0.3)
    ax.set_yscale('log')
    
    # Plot 3: Final performance comparison
    ax = axes[3]
    
    method_names = list(results_adaptive.keys())
    final_losses = [results_adaptive[name][1]['f'][-1] for name in method_names]
    iterations = [len(results_adaptive[name][1]['f'])-1 for name in method_names]
    
    x_pos = np.arange(len(method_names))
    bars = ax.bar(x_pos, final_losses, color=colors_adaptive[:len(method_names)], alpha=0.7)
    
    # Display iteration count on top of bars
    for i, (bar, iters) in enumerate(zip(bars, iterations)):
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height + height*0.05,
                f'{iters}it', ha='center', va='bottom', fontsize=9)
    
    ax.set_xlabel('Algorithm')
    ax.set_ylabel('Final Loss')
    ax.set_title('Final Loss Comparison')
    ax.set_xticks(x_pos)
    ax.set_xticklabels(method_names, rotation=45)
    ax.grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.show()

# Call the visualization function
plot_adaptive_methods_comparison()

In [ ]:
# Visualization 1: Contour plots with optimization paths for quadratic functions

def plot_optimization_paths():
    """Visualize optimization paths on quadratic functions"""
    
    # Check if results are available
    if 'results_quadratic' not in globals() or not results_quadratic:
        print("Error: Algorithm results not available. Please run the algorithm comparison cell first.")
        return
    
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes = axes.flatten()
    
    # Prepare visualization for condition number 10 function
    x_range = np.linspace(-6, 6, 100)
    y_range = np.linspace(-6, 6, 100)
    X, Y = np.meshgrid(x_range, y_range)
    Z1 = 0.5 * (Q1[0,0] * X**2 + Q1[1,1] * Y**2) + b1[0] * X + b1[1] * Y
    
    # Prepare visualization for condition number 1000 function  
    x_range2 = np.linspace(-4, 4, 100)
    y_range2 = np.linspace(-2, 2, 100)
    X2, Y2 = np.meshgrid(x_range2, y_range2)
    Z2 = 0.5 * (Q2[0,0] * X2**2 + Q2[1,1] * Y2**2) + b2[0] * X2 + b2[1] * Y2
    
    # Starting points
    start_points = [np.array([4.0, -3.0]), np.array([3.0, -2.0])]
    
    # Plot 0: Condition number 10, various algorithms
    ax = axes[0]
    ax.contour(X, Y, Z1, levels=20, colors='gray', alpha=0.6, linewidths=0.8)
    ax.contourf(X, Y, Z1, levels=20, alpha=0.3, cmap='viridis')
    
    methods_to_plot = ['GD', 'BFGS', 'Momentum', 'Nesterov']
    colors_plot = ['red', 'blue', 'green', 'purple']
    markers = ['o', 's', '^', 'D']
    
    for i, method in enumerate(methods_to_plot):
        if method in results_quadratic:
            path = np.array(results_quadratic[method][1]['x'])
            # Limit path length for visualization
            path = path[:min(50, len(path))]
            ax.plot(path[:, 0], path[:, 1], color=colors_plot[i], marker=markers[i], 
                   markersize=3, alpha=0.8, linewidth=2, label=method)
    
    ax.plot(analytical_opt1[0], analytical_opt1[1], 'k*', markersize=12, label='Optimum')
    ax.plot(start_points[0][0], start_points[0][1], 'ko', markersize=8, label='Start')
    ax.set_title(f'Optimization Paths (κ={np.linalg.cond(Q1):.0f})', fontweight='bold')
    ax.legend(fontsize=9)
    ax.grid(True, alpha=0.3)
    ax.set_aspect('equal')
    
    # Plot 1: Condition number 1000, gradient descent comparison
    ax = axes[1]
    ax.contour(X2, Y2, Z2, levels=20, colors='gray', alpha=0.6, linewidths=0.8)
    ax.contourf(X2, Y2, Z2, levels=20, alpha=0.3, cmap='viridis')
    
    # Run gradient descent on ill-conditioned function for visualization
    try:
        x_gd_ill, hist_gd_ill = gradient_descent(f2, grad_f2, start_points[1], learning_rate=0.001, max_iter=200)
        path_ill = np.array(hist_gd_ill['x'])
        path_ill = path_ill[:min(100, len(path_ill))]  # Limit for visualization
        
        ax.plot(path_ill[:, 0], path_ill[:, 1], 'red', marker='o', markersize=2, 
               alpha=0.7, linewidth=1, label='GD')
    except Exception as e:
        print(f"Warning: Could not generate ill-conditioned visualization: {e}")
    
    ax.plot(analytical_opt2[0], analytical_opt2[1], 'k*', markersize=12, label='Optimum')
    ax.plot(start_points[1][0], start_points[1][1], 'ko', markersize=8, label='Start')
    ax.set_title(f'Ill-Conditioned Function (κ={np.linalg.cond(Q2):.0f})', fontweight='bold')
    ax.legend(fontsize=9)
    ax.grid(True, alpha=0.3)
    ax.set_aspect('equal')
    
    # Plot 2: Step size visualization
    ax = axes[2]
    step_plotted = False
    for i, method in enumerate(['GD', 'BFGS', 'Momentum', 'Nesterov']):
        if method in results_quadratic:
            step_sizes = results_quadratic[method][1]['step_size']
            if step_sizes:  # Check if step_size data exists
                step_data = step_sizes[:min(50, len(step_sizes))]
                if len(step_data) > 1:  # Only plot if we have meaningful data
                    ax.plot(step_data, color=colors_plot[i], 
                           linewidth=2, label=method, alpha=0.8)
                    step_plotted = True
    
    if step_plotted:
        ax.set_xlabel('Iteration')
        ax.set_ylabel('Step Size')
        ax.set_title('Step Size Evolution')
        ax.legend()
        ax.grid(True, alpha=0.3)
        ax.set_yscale('log')
    else:
        ax.text(0.5, 0.5, 'Step size data\nnot available', ha='center', va='center', 
                transform=ax.transAxes, fontsize=12)
        ax.set_title('Step Size Evolution')
    
    # Plot 3: Function value convergence curves (condition number 10)
    ax = axes[3]
    for i, method in enumerate(methods_to_plot):
        if method in results_quadratic:
            f_values = results_quadratic[method][1]['f']
            # Difference from optimal value
            f_diff = np.array(f_values) - f1(analytical_opt1)
            f_diff = np.maximum(f_diff, 1e-16)  # Clipping for log
            ax.semilogy(f_diff, color=colors_plot[i], linewidth=2, label=method)
    
    ax.set_xlabel('Iteration')
    ax.set_ylabel('f(x) - f*')
    ax.set_title('Convergence: f(x) - f*')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # Plot 4: Gradient norm evolution
    ax = axes[4]
    for i, method in enumerate(methods_to_plot):
        if method in results_quadratic:
            grad_norms = results_quadratic[method][1]['grad_norm']
            ax.semilogy(grad_norms, color=colors_plot[i], linewidth=2, label=method)
    
    ax.set_xlabel('Iteration')
    ax.set_ylabel('||∇f(x)||')
    ax.set_title('Gradient Norm Evolution')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # Plot 5: Computation time comparison
    ax = axes[5]
    methods_time = []
    times = []
    iterations = []
    
    for method_name in methods_to_plot:
        if method_name in results_quadratic:
            hist = results_quadratic[method_name][1]
            methods_time.append(method_name)
            times.append(hist['time'][-1])
            iterations.append(len(hist['f'])-1)
    
    if methods_time:  # Only create plot if we have data
        x_pos = np.arange(len(methods_time))
        bars = ax.bar(x_pos, times, color=colors_plot[:len(methods_time)], alpha=0.7)
        
        # Display iteration count on top of bars
        for i, (bar, iters) in enumerate(zip(bars, iterations)):
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height + height*0.05,
                    f'{iters}it', ha='center', va='bottom', fontsize=9)
        
        ax.set_xlabel('Algorithm')
        ax.set_ylabel('Time (seconds)')
        ax.set_title('Computation Time Comparison')
        ax.set_xticks(x_pos)
        ax.set_xticklabels(methods_time, rotation=45)
        ax.grid(True, alpha=0.3, axis='y')
    else:
        ax.text(0.5, 0.5, 'Timing data\nnot available', ha='center', va='center', 
                transform=ax.transAxes, fontsize=12)
        ax.set_title('Computation Time Comparison')
    
    plt.tight_layout()
    plt.show()

# Call the visualization function
plot_optimization_paths()

In [ ]:
# Visualization 2: Learning rate and convergence speed analysis

def plot_learning_rate_analysis():
    """Detailed analysis of learning rate impact and convergence speed"""
    
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    
    # Various learning rates experiment
    learning_rates = [0.05, 0.1, 0.15, 0.2]
    x0 = np.array([3.0, -2.0])
    
    # Plot 0: Convergence curves by learning rate (condition number 10)
    ax = axes[0, 0]
    for i, lr in enumerate(learning_rates):
        try:
            x_opt, hist = gradient_descent(f1, grad_f1, x0, learning_rate=lr, max_iter=100)
            f_diff = np.array(hist['f']) - f1(analytical_opt1)
            f_diff = np.maximum(f_diff, 1e-16)
            ax.semilogy(f_diff, color=colors[i], linewidth=2, label=f'α={lr}')
        except Exception as e:
            print(f"Warning: Could not plot learning rate {lr}: {e}")
    
    ax.set_xlabel('Iteration')
    ax.set_ylabel('f(x) - f*')
    ax.set_title('Learning Rate Effect (κ=10)')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # Plot 1: Convergence curves by learning rate (condition number 1000)
    ax = axes[0, 1]
    learning_rates_bad = [0.0005, 0.001, 0.0015, 0.002]
    for i, lr in enumerate(learning_rates_bad):
        try:
            x_opt, hist = gradient_descent(f2, grad_f2, x0, learning_rate=lr, max_iter=1000)
            f_diff = np.array(hist['f']) - f2(analytical_opt2)
            f_diff = np.maximum(f_diff, 1e-16)
            # Length limit
            max_len = min(500, len(f_diff))
            ax.semilogy(f_diff[:max_len], color=colors[i], linewidth=2, label=f'α={lr}')
        except Exception as e:
            print(f"Warning: Could not plot learning rate {lr}: {e}")
    
    ax.set_xlabel('Iteration')
    ax.set_ylabel('f(x) - f*')  
    ax.set_title('Learning Rate Effect (κ=1000)')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # Plot 2: Step size changes (Armijo vs Wolfe)
    ax = axes[1, 0]
    
    try:
        # Armijo vs Wolfe step size comparison
        x_armijo, hist_armijo = gradient_descent_with_line_search(f1, grad_f1, x0, 'armijo', max_iter=50)
        x_wolfe, hist_wolfe = gradient_descent_with_line_search(f1, grad_f1, x0, 'wolfe', max_iter=50)
        
        ax.plot(hist_armijo['step_size'], 'b-', linewidth=2, label='Armijo', alpha=0.8)
        ax.plot(hist_wolfe['step_size'], 'r-', linewidth=2, label='Wolfe', alpha=0.8)
        
        ax.set_xlabel('Iteration')
        ax.set_ylabel('Step Size (α)')
        ax.set_title('Line Search Comparison')
        ax.legend()
        ax.grid(True, alpha=0.3)
    except Exception as e:
        print(f"Warning: Could not generate line search comparison: {e}")
        ax.text(0.5, 0.5, 'Line search data\nnot available', ha='center', va='center', 
                transform=ax.transAxes, fontsize=12)
        ax.set_title('Line Search Comparison')
    
    # Plot 3: Algorithm performance comparison
    ax = axes[1, 1]
    
    # Check if we have quadratic results
    if 'results_quadratic' in globals() and results_quadratic:
        methods_time = []
        times = []
        iterations = []
        
        for method_name in ['GD', 'BFGS', 'L-BFGS', 'Momentum', 'Nesterov']:
            if method_name in results_quadratic:
                hist = results_quadratic[method_name][1]
                methods_time.append(method_name)
                times.append(hist['time'][-1])
                iterations.append(len(hist['f'])-1)
        
        if methods_time:
            x_pos = np.arange(len(methods_time))
            bars = ax.bar(x_pos, times, color=colors[:len(methods_time)], alpha=0.7)
            
            # Display iteration count on top of bars
            for i, (bar, iters) in enumerate(zip(bars, iterations)):
                height = bar.get_height()
                ax.text(bar.get_x() + bar.get_width()/2., height + height*0.05,
                        f'{iters}it', ha='center', va='bottom', fontsize=9)
            
            ax.set_xlabel('Algorithm')
            ax.set_ylabel('Time (seconds)')
            ax.set_title('Performance Comparison')
            ax.set_xticks(x_pos)
            ax.set_xticklabels(methods_time, rotation=45)
            ax.grid(True, alpha=0.3, axis='y')
        else:
            ax.text(0.5, 0.5, 'Performance data\nnot available', ha='center', va='center', 
                    transform=ax.transAxes, fontsize=12)
            ax.set_title('Performance Comparison')
    else:
        ax.text(0.5, 0.5, 'Performance data\nnot available', ha='center', va='center', 
                transform=ax.transAxes, fontsize=12)
        ax.set_title('Performance Comparison')
    
    plt.tight_layout()
    plt.show()

# Call the visualization function
plot_learning_rate_analysis()

In [ ]:
# Performance summary table and key observations

def create_summary_table():
    """Generate optimization algorithm performance summary table"""
    
    # Check if results are available
    if 'results_quadratic' not in globals() or not results_quadratic:
        print("Error: Quadratic algorithm results not available")
        return [], []
    
    if 'results_adaptive' not in globals() or not results_adaptive:
        print("Error: Adaptive algorithm results not available") 
        return [], []
    
    # Collect quadratic function results (condition number 10)
    summary_data = []
    
    for method_name, (x_opt, hist) in results_quadratic.items():
        final_f = hist['f'][-1]
        final_grad_norm = hist['grad_norm'][-1]
        iterations = len(hist['f']) - 1
        total_time = hist['time'][-1]
        avg_step = np.mean(hist['step_size']) if hist['step_size'] else 0
        
        summary_data.append({
            'Algorithm': method_name,
            'Final f': f"{final_f:.2e}",
            'Final ||∇f||': f"{final_grad_norm:.2e}", 
            'Iterations': iterations,
            'Time (s)': f"{total_time:.4f}",
            'Avg Step': f"{avg_step:.4f}"
        })
    
    # Add adaptive learning rate results (logistic regression)
    adaptive_data = []
    for method_name, (w_opt, hist) in results_adaptive.items():
        final_loss = hist['f'][-1]
        final_grad_norm = hist['grad_norm'][-1] 
        iterations = len(hist['f']) - 1
        total_time = hist['time'][-1]
        avg_step = np.mean(hist['step_size']) if hist['step_size'] else 0
        
        adaptive_data.append({
            'Algorithm': method_name,
            'Final Loss': f"{final_loss:.4f}",
            'Final ||∇f||': f"{final_grad_norm:.2e}",
            'Iterations': iterations,
            'Time (s)': f"{total_time:.4f}",
            'Avg Step': f"{avg_step:.4f}"
        })
    
    return summary_data, adaptive_data

# Create and display summary tables
try:
    summary_quadratic, summary_adaptive = create_summary_table()
    
    if summary_quadratic and summary_adaptive:
        print("Optimization Algorithm Performance Summary")
        print("="*80)

        # Quadratic function summary table
        print(f"\nQuadratic Function Optimization Results (condition number {np.linalg.cond(Q1):.0f})")
        print("-"*80)
        df_quad = pd.DataFrame(summary_quadratic)
        df_quad_sorted = df_quad.sort_values('Iterations').reset_index(drop=True)
        print(df_quad_sorted.to_string(index=False))

        # Adaptive learning rate summary table 
        print(f"\nAdaptive Learning Rate Methods Results (Logistic Regression)")
        print("-"*80)
        df_adaptive = pd.DataFrame(summary_adaptive)
        df_adaptive_sorted = df_adaptive.sort_values('Iterations').reset_index(drop=True)
        print(df_adaptive_sorted.to_string(index=False))

        # Condition number impact comparison
        print(f"\nCondition Number Impact Comparison")
        print("-"*40)
        print(f"Condition number {np.linalg.cond(Q1):.0f} vs {np.linalg.cond(Q2):.0f}")

        # Check if hist_gd2 exists for comparison
        if 'hist_gd2' in globals() and 'hist_gd' in globals():
            gd_cond10_iters = len(hist_gd['f']) - 1
            gd_cond1000_iters = len(hist_gd2['f']) - 1
            print(f"Gradient descent iterations: {gd_cond10_iters} vs {gd_cond1000_iters}")
            print(f"Convergence speed difference: {gd_cond1000_iters/gd_cond10_iters:.1f}x slower")
        else:
            print("Note: Detailed condition number comparison requires running the full algorithm comparison.")

        print("\nPerformance summary completed successfully")
    else:
        print("Error: Could not generate performance summary tables")
        
except Exception as e:
    print(f"Error creating performance summary: {e}")
    print("Please ensure all algorithm comparison cells have been executed successfully.")

In [ ]:
# Master execution cell: Run all analysis and visualizations

def run_complete_analysis():
    """Execute complete optimization algorithm analysis and visualization"""
    
    print("Starting complete optimization algorithm analysis...")
    print("="*60)
    
    # Step 1: Verify basic setup
    try:
        print("Step 1: Verifying basic functions...")
        test_f = f1(np.array([1.0, 1.0]))
        print(f"Quadratic functions working (test: f1([1,1]) = {test_f:.4f})")
    except Exception as e:
        print(f"Basic function error: {e}")
        return False
    
    # Step 2: Test line search methods
    try:
        print("\nStep 2: Testing line search methods...")
        test_line_search_methods()
    except Exception as e:
        print(f"Line search test error: {e}")
    
    # Step 3: Run algorithm comparisons
    try:
        print("\nStep 3: Running algorithm comparisons...")
        print("This may take a moment...")
        
        # Algorithm comparison code
        x0 = np.array([4.0, -3.0])

        # BFGS vs L-BFGS comparison
        print("Running quasi-Newton methods...")
        x_bfgs, hist_bfgs = bfgs(f1, grad_f1, x0, max_iter=50)
        x_lbfgs, hist_lbfgs = lbfgs(f1, grad_f1, x0, max_iter=50)

        # Momentum methods comparison  
        print("Running momentum methods...")
        x_gd, hist_gd = gradient_descent(f1, grad_f1, x0, learning_rate=0.1, max_iter=100)
        x_mom, hist_mom = momentum(f1, grad_f1, x0, learning_rate=0.1, beta=0.9, max_iter=100)  
        x_nest, hist_nest = nesterov(f1, grad_f1, x0, learning_rate=0.1, beta=0.9, max_iter=100)

        # Store results
        globals()['results_quadratic'] = {
            "BFGS": (x_bfgs, hist_bfgs),
            "L-BFGS": (x_lbfgs, hist_lbfgs), 
            "GD": (x_gd, hist_gd),
            "Momentum": (x_mom, hist_mom),
            "Nesterov": (x_nest, hist_nest)
        }

        print("Algorithm comparison completed")
        
    except Exception as e:
        print(f"Algorithm comparison had issues: {e}")
    
    # Step 4: Run adaptive methods
    try:
        print("\nStep 4: Running adaptive learning rate methods...")
        w0 = np.random.randn(5) * 0.1
        
        print("Running Adagrad...")
        w_adagrad, hist_adagrad = adagrad(logistic_f, logistic_grad_f, w0, learning_rate=0.1, max_iter=500)
        
        print("Running RMSProp...")  
        w_rmsprop, hist_rmsprop = rmsprop(logistic_f, logistic_grad_f, w0, learning_rate=0.01, max_iter=500)
        
        print("Running Adam...")
        w_adam, hist_adam = adam(logistic_f, logistic_grad_f, w0, learning_rate=0.01, max_iter=500)
        
        print("Running Adadelta...")
        w_adadelta, hist_adadelta = adadelta(logistic_f, logistic_grad_f, w0, max_iter=500)
        
        # Store adaptive results
        globals()['results_adaptive'] = {
            'Adagrad': (w_adagrad, hist_adagrad),
            'RMSProp': (w_rmsprop, hist_rmsprop),  
            'Adam': (w_adam, hist_adam),
            'Adadelta': (w_adadelta, hist_adadelta),
        }
        
        print("Adaptive methods completed")
        
    except Exception as e:
        print(f"Adaptive methods had issues: {e}")
    
    # Step 5: Generate visualizations
    print("\nStep 5: Generating visualizations...")
    
    try:
        print("Creating main optimization paths visualization...")
        plot_optimization_paths()
        print("Main visualization completed")
    except Exception as e:
        print(f"Main visualization error: {e}")
    
    try:
        print("Creating learning rate analysis...")
        plot_learning_rate_analysis()
        print("Learning rate analysis completed")
    except Exception as e:
        print(f"Learning rate analysis error: {e}")
    
    try:
        print("Creating adaptive methods comparison...")
        plot_adaptive_methods_comparison()
        print("Adaptive methods visualization completed")
    except Exception as e:
        print(f"Adaptive methods visualization error: {e}")
    
    # Step 6: Generate summary
    print("\nStep 6: Generating performance summary...")
    try:
        summary_quadratic, summary_adaptive = create_summary_table()

        if summary_quadratic and summary_adaptive:
            print("\n" + "="*80)
            print("FINAL PERFORMANCE SUMMARY")
            print("="*80)
            
            # Display tables...
            df_quad = pd.DataFrame(summary_quadratic)
            df_adaptive = pd.DataFrame(summary_adaptive)
            
            print("\nQuadratic Function Results:")
            print(df_quad.to_string(index=False))
            
            print("\nAdaptive Methods Results:")  
            print(df_adaptive.to_string(index=False))
            
            print("\nAnalysis completed successfully!")
    except Exception as e:
        print(f"Summary generation error: {e}")
    
    print("\nComplete analysis finished!")
    return True

# Run the complete analysis
run_complete_analysis()

## Key Observations and Conclusions

### Main Findings

1. **Impact of condition number**: The larger the matrix condition number, the significantly slower the convergence of gradient descent. Functions with condition number 1000 showed convergence over 10 times slower.

2. **Superiority of second-order methods**: BFGS and L-BFGS show very fast convergence on quadratic functions. BFGS theoretically converges in finite steps for quadratic functions.

3. **Effect of momentum**: Momentum and Nesterov acceleration show improved convergence over gradient descent, but are not as effective as second-order methods on quadratic functions.

4. **Adaptive learning rates**: Adam and RMSProp demonstrate stable performance on nonlinear problems like logistic regression.

5. **Importance of line search**: Line search using Armijo and Wolfe conditions provides more stable and faster convergence than fixed learning rates.

### Practical Recommendations

- **Quadratic functions**: Use BFGS or L-BFGS
- **Large-scale problems**: Use L-BFGS or Adam
- **Neural network training**: Use Adam, RMSProp, or adaptive momentum
- **Ill-conditioned problems**: Use quasi-Newton methods with line search

## Appendix: Nonlinear Example

As a reference, we briefly present optimization experiments on a nonlinear test function.

Test function: $f(x, y) = (1-x)^2 + 100(y-x^2)^2$

This function is challenging to optimize due to its nonlinearity and narrow valley structure.

In [ ]:
# Appendix: Nonlinear function experiment

def nonlinear_test_function(x):
    """Nonlinear test function"""
    return (1 - x[0])**2 + 100 * (x[1] - x[0]**2)**2

def nonlinear_test_grad(x):
    """Gradient of nonlinear test function"""
    grad_x = -2*(1 - x[0]) - 400*x[0]*(x[1] - x[0]**2)
    grad_y = 200*(x[1] - x[0]**2)
    return np.array([grad_x, grad_y])

def nonlinear_test_hess(x):
    """Hessian of nonlinear test function"""
    h11 = 2 + 1200*x[0]**2 - 400*x[1]
    h12 = h21 = -400*x[0]
    h22 = 200
    return np.array([[h11, h12], [h21, h22]])

print("Appendix: Nonlinear Function Optimization")
print("="*40)

x0_nonlinear = np.array([-1.0, 1.0])
print(f"Starting point: {x0_nonlinear}")
print(f"Global optimum: [1.0, 1.0]")
print(f"Optimal value: 0.0")
print()

# Test with several algorithms
methods_nonlinear = [
    ('GD (lr=0.001)', lambda: gradient_descent(nonlinear_test_function, nonlinear_test_grad, x0_nonlinear, 
                                             learning_rate=0.001, max_iter=10000)),
    ('BFGS', lambda: bfgs(nonlinear_test_function, nonlinear_test_grad, x0_nonlinear, max_iter=100)),
    ('Adam (lr=0.01)', lambda: adam(nonlinear_test_function, nonlinear_test_grad, x0_nonlinear, 
                                  learning_rate=0.01, max_iter=1000)),
]

for name, method in methods_nonlinear:
    x_opt, hist = method()
    final_error = np.linalg.norm(x_opt - np.array([1.0, 1.0]))
    final_f = nonlinear_test_function(x_opt)
    iterations = len(hist['f']) - 1
    print(f"{name:15}: Final error {final_error:.3f}, Function value {final_f:.3f}, Iterations {iterations:4d}")

# Simple visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

# Nonlinear function contours
x_range = np.linspace(-2, 2, 100)
y_range = np.linspace(-1, 3, 100) 
X, Y = np.meshgrid(x_range, y_range)
Z = (1 - X)**2 + 100*(Y - X**2)**2

# Display contours on log scale
ax1.contour(X, Y, np.log(Z + 1), levels=20, colors='gray', alpha=0.6)
ax1.contourf(X, Y, np.log(Z + 1), levels=20, alpha=0.4, cmap='viridis')

# Display BFGS path (recalculate)
x_bfgs_nonlinear, hist_bfgs_nonlinear = bfgs(nonlinear_test_function, nonlinear_test_grad, x0_nonlinear, max_iter=50)
path = np.array(hist_bfgs_nonlinear['x'])
ax1.plot(path[:, 0], path[:, 1], 'ro-', markersize=4, linewidth=2, alpha=0.8, label='BFGS')
ax1.plot(x0_nonlinear[0], x0_nonlinear[1], 'ko', markersize=8, label='Start')
ax1.plot(1, 1, 'k*', markersize=12, label='Optimum')
ax1.set_title('Nonlinear Function (log scale)')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Convergence curve
ax2.semilogy(hist_bfgs_nonlinear['f'], 'b-', linewidth=2, label='BFGS')
ax2.set_xlabel('Iteration')
ax2.set_ylabel('Function Value (log scale)')
ax2.set_title('BFGS Convergence on Nonlinear Function')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nExcellent performance of BFGS on nonlinear functions is confirmed.")